In [1]:
import os

In [2]:
from glob import glob

In [2]:
import re
import csv
import isodate
import simplejson as json
import numpy as np

In [3]:
from matplotlib import pyplot as plt # import submodules (not the while module) + also alias
# cell magic that renders gfx in the notebook. Try %matplotlib notebook
%matplotlib inline
from itertools import groupby

/Users/Mandi/anaconda/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [21]:
alias say echo "I want you to say %l"

In [22]:
alias saytwice echo hi %s hello %s

In [23]:
saytwice boop beep

hi boop hello beep


In [24]:
alias show echo

In [25]:
STRING='A Python string'

In [27]:
show $STRING

A Python string


In [28]:
show $$PATH

/Users/Mandi/anaconda/bin:/Users/Mandi/anaconda/bin:/usr/local/bin:/Users/Mandi/Documents/google-cloud-sdk/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin


In [30]:
%alias

Total number of aliases: 16


[('bracket', 'echo "I want you to say %l"'),
 ('cat', 'cat'),
 ('cp', 'cp'),
 ('ldir', 'ls -F -G -l %l | grep /$'),
 ('lf', 'ls -F -l -G %l | grep ^-'),
 ('lk', 'ls -F -l -G %l | grep ^l'),
 ('ll', 'ls -F -l -G'),
 ('ls', 'ls -F -G'),
 ('lx', 'ls -F -l -G %l | grep ^-..x'),
 ('mkdir', 'mkdir'),
 ('mv', 'mv'),
 ('rm', 'rm'),
 ('rmdir', 'rmdir'),
 ('say', 'echo "I want you to say %l"'),
 ('saytwice', 'echo hi %s hello %s'),
 ('show', 'echo')]

In [31]:
## Print the type
def pto(s):
    print(type(s))

In [32]:
pto('hello')

<class 'str'>


In [70]:
import os 

DATADIRECTORY = 'Reports_Run_3299'
DATAFILE = os.path.join(DATADIRECTORY, 'EchoComLog.txt')

list_of_files = os.listdir(DATADIRECTORY)
pto(list_of_files)
getext = lambda s:os.path.splitext(s)[1] # lambda are anonynmous functions that you can define on the fly and forget about when you're done
list_of_files.sort(key=getext) # Key = (some function) calls a function to be applied to the elements before they are sorted
                               # Getext is the function that splits the text 
list(groupby(list_of_files, getext)) # Groupby allows you to group data into buckets based on a variable

<class 'list'>


[('.CSV', <itertools._grouper at 0x109c02f60>),
 ('.XML', <itertools._grouper at 0x109c02ef0>),
 ('.csv', <itertools._grouper at 0x109c02e10>),
 ('.run', <itertools._grouper at 0x109c02cc0>),
 ('.txt', <itertools._grouper at 0x109c02be0>),
 ('.xml', <itertools._grouper at 0x109c02b70>)]

In [71]:
from glob import glob
xml_files = glob(DATADIRECTORY+'/*.XML')
xml_files[1:5]

['Reports_Run_3299/E5XX-1111_Print_1486627744_Exceptions.XML',
 'Reports_Run_3299/E5XX-1111_Survey_Source[10](AAAOZPZ).XML',
 'Reports_Run_3299/E5XX-1111_Survey_Source[11](AAAOZP0).XML',
 'Reports_Run_3299/E5XX-1111_Survey_Source[12](AAAOZP1).XML']

In [72]:
dict = {}
dict['cat'] = 'meow'
dict['dog'] = 'woof'
print(dict)
print(len(dict.keys()))

{'cat': 'meow', 'dog': 'woof'}
2


In [80]:
# "regular expressions"
# SEE http://pythex.org/
# SEE https://developers.google.com/edu/python/regular-expressions
well_begin = '.*DEBUG.*Well\s*([a-zA-Z]+)(\d+)\s*begin.*' # captures well letter and number: P16 -> P,16
# well_end = '.*INFO.*Well.*Transfer Complete.*' # just ends it for now
well_end = '.*INFO.*Well\s*<([a-zA-Z]+)(\d+)\s*->\s*([a-zA-Z]+)(\d+)\W*([0-9\.]+)\s*(\w+)\s*>\s*Transfer Complete.*' # fromLetter, fromNumber, toLetter, toNumber, vol, units
well_data_line = '\s*(.+)\:\s*([0-9\.]+)\s*(\S+)' # Property key, value, units

# this is what we'd like to avoid
with open(DATAFILE, 'r') as fh:
    in_well_transfer_state = False
    current_active_well = None
    well_info = dict()
    for line in fh: # file (text mode) opens as an iterable, split by lines
        linedata = line.split()
        if not in_well_transfer_state:
            match = re.match(well_begin, line)
            if match:
                in_well_transfer_state = True
                current_active_well = tuple([match.group(1),int(match.group(2))])
                well_info[current_active_well] = dict()
                continue # go back to beginning of for loop
        if in_well_transfer_state:
            match = re.match(well_end, line)
            if match:
                well_from = (match.group(1),int(match.group(2)))
                well_to = (match.group(3),int(match.group(4)))
                volume = float(match.group(5))
                unit = match.group(6)
                well_info[current_active_well]['Transfer'] = {'from':well_from, 'to':well_to, 'volume':volume, 'unit':unit}
                in_well_transfer_state = False
                current_active_well = None # just to shake out errors
                continue # continue at the top of the for loop
            another_match = re.match(well_data_line, line)
            if another_match:
                prop = another_match.group(1)
                value = float(another_match.group(2))
                unit = another_match.group(3)
                well_info[current_active_well][prop] = {'value': value, 'unit': unit}
                
print('Loaded {} wells'.format(len(well_info.keys())))

TypeError: 'dict' object is not callable